### Step 5
* predict un and test
* ouput fname,softmax CSV

In [28]:
import numpy as np
from random import shuffle
from math import log, floor
import pandas as pd
import tensorflow as tf
import tensorboard as tb
from keras import backend as K
from keras.models import *
from keras.layers import *
from keras.activations import *
from keras.callbacks import *
from keras.utils import *
from keras.layers.advanced_activations import *
from collections import Counter
from keras import *
from keras.engine.topology import *
from keras.optimizers import *
import keras
# import pandas as pd
import glob
from sklearn.semi_supervised import *
import pickle
from keras.applications import *
from keras.preprocessing.image import *
from keras.losses import mse, binary_crossentropy
import pandas as pd # data frame
import numpy as np # matrix math
from scipy.io import wavfile # reading the wavfile
from sklearn.utils import shuffle # shuffling of data
from random import sample # random selection
from tqdm import tqdm # progress bar
import matplotlib.pyplot as plt # to view graphs
import wave
from math import log, floor
# audio processing
from scipy import signal # audio processing
from scipy.fftpack import dct
import librosa # library for audio processing
import numpy as np
import pandas as pd
from sklearn.decomposition import *
from sklearn.cluster import KMeans
import sys, os
import random,math
from tqdm import tqdm ##
from xgboost.sklearn import XGBClassifier
from sklearn.utils import shuffle # shuffling of data
from random import sample # random selection
from tqdm import tqdm # progress bar
# audio processing
from scipy import signal # audio processing
from scipy.fftpack import dct
import librosa # library for audio processing
import xgboost as xgb
import lightgbm as lgb
import catboost as ctb
from keras.utils import *
from sklearn.ensemble import *
import pickle
from bayes_opt import BayesianOptimization
from logHandler import Logger
from utils import readCSV, getPath, writePickle,readPickle
from keras.regularizers import l2
from keras.callbacks import History ,ModelCheckpoint, EarlyStopping

import resnet
from random_eraser import get_random_eraser
from mixup_generator import MixupGenerator

每個fold只能留下一個model

In [29]:
def output_softmax(feature_type,test_or_un,base_model,co_or_self):
    if not os.path.exists('feature/'+feature_type+'/'+co_or_self+'/unverified/'):
        os.makedirs('feature/'+feature_type+'/'+co_or_self+'/unverified/') # unverified儲存資料夾
    if not os.path.exists('feature/'+feature_type+'/'+co_or_self+'/test/'):
        os.makedirs('feature/'+feature_type+'/'+co_or_self+'/test/') # test儲存資料夾

    if test_or_un =='un':
        semi = np.load('feature/'+feature_type+'/X_unverified.npy') # predict unverified
        fname = pd.DataFrame(np.load('feature/'+feature_type+'/fname_unverified.npy')) #df
    elif test_or_un == 'test':
        semi = np.load('feature/'+feature_type+'/X_test.npy') #predict test
        fname = pd.read_csv('data/sample_submission.csv').filter(['fname']) #df
    else:
        semi = np.load('feature/'+feature_type+'/X_test.npy') #predict test
        fname = pd.read_csv('data/sample_submission.csv').filter(['fname']) #df
        semi_un = np.load('feature/'+feature_type+'/X_unverified.npy') # predict unverified
        fname_un = pd.DataFrame(np.load('feature/'+feature_type+'/fname_unverified.npy')) #df

    for fold in tqdm(range(num_fold)):
        model = load_model(glob.glob(base_model+'**_fold'+str(fold)+'_**')[0])
        ans = model.predict(semi)
        try:
            ans_un = model.predict(semi_un)
            ans_un = pd.DataFrame(ans_un)
            df_un = pd.merge(fname_un,ans_un,how='inner',right_index=True,left_index=True)
            df_un = df_un.rename(columns={'0_x':'fname','0_y':'0'})
        except:
            pass
        ans = pd.DataFrame(ans) #df
        df = pd.merge(fname,ans,how='inner',right_index=True,left_index=True)
        df = df.rename(columns={'0_x':'fname','0_y':'0'})
#         print(len(df))
        if test_or_un =='un':
            df.to_csv('feature/'+feature_type+'/'+co_or_self+'/unverified/leo_resnet_'+feature_type+'_unverified_'+str(fold+1)+'.csv',index=False)
        elif test_or_un=='test':
            df.to_csv('feature/'+feature_type+'/'+co_or_self+'/test/leo_resnet_'+feature_type+'_test_'+str(fold+1)+'.csv',index=False)
        else:
            df.to_csv('feature/'+feature_type+'/'+co_or_self+'/test/leo_resnet_'+feature_type+'_test_'+str(fold+1)+'.csv',index=False)
            df_un.to_csv('feature/'+feature_type+'/'+co_or_self+'/unverified/leo_resnet_'+feature_type+'_unverified_'+str(fold+1)+'.csv',index=False)

    return df

In [30]:
feature_type = ['mfcc6','mfcc7']
num_fold = 10
test_or_un = ['both'] # un or test or both
co_or_self = 'selftrain' # cotrain or selftrain


for fea in feature_type:
    for tou in test_or_un:
        base_model = 'model/'+fea+'/'
        df = output_softmax(feature_type=fea,test_or_un=tou,base_model=base_model,co_or_self=co_or_self)
        print('===finish:',tou,fea,len(df),'===')

df        

100%|██████████| 10/10 [11:27<00:00, 68.77s/it]


===finish: both mfcc6 9400 ===


100%|██████████| 10/10 [39:47<00:00, 238.71s/it]

===finish: both mfcc7 9400 ===


**__append un and test__**

In [32]:
feature_type = ['mfcc6','mfcc7']
num_fold = 10
# test_or_un = ['both'] # un or test or both
co_or_self = 'selftrain' # cotrain or selftrain

In [36]:

for fea in feature_type:
    if not os.path.exists('feature/'+fea+'/'+co_or_self+'/combine/'):
        os.makedirs('feature/'+fea+'/'+co_or_self+'/combine/') # unverified儲存資料夾
    for fold in range(num_fold):
        df_un = pd.read_csv('feature/'+fea+'/'+co_or_self+'/unverified/leo_resnet_'+fea+'_unverified_'+str(fold+1)+'.csv')
        df = pd.read_csv('feature/'+fea+'/'+co_or_self+'/test/leo_resnet_'+fea+'_test_'+str(fold+1)+'.csv') #test
        df = df_un.append(df)
        df.to_csv('feature/'+fea+'/'+co_or_self+'/combine/leo_resnet_'+fea+'_semi_'+str(fold+1)+'.csv',index=False)
    print('===finish:',fea,len(df),'===')
df

===finish: mfcc6 15163 ===
===finish: mfcc7 15163 ===


,fname,0,1,2,3,4,5,6,7,8,...,31,32,33,34,35,36,37,38,39,40
0,00044347.wav,0.544194,0.015410,0.085462,0.005777,0.058388,0.000051,0.002902,0.026951,0.000016,...,0.000398,0.001448,0.000681,0.004137,0.002394,0.000076,0.001254,0.007413,0.028340,0.000127
1,002d256b.wav,0.008712,0.000979,0.000365,0.000317,0.000013,0.010587,0.009455,0.000898,0.000712,...,0.006647,0.002561,0.001962,0.000389,0.005040,0.071015,0.000113,0.011196,0.010737,0.049264
2,003b91e8.wav,0.000143,0.010829,0.017804,0.000015,0.844686,0.000017,0.000138,0.001281,0.000011,...,0.000069,0.000033,0.000080,0.000518,0.000028,0.000018,0.000038,0.000441,0.001817,0.000020
3,004ad66f.wav,0.000218,0.225169,0.054935,0.000047,0.001256,0.000261,0.000858,0.421226,0.000304,...,0.000299,0.000299,0.000449,0.000103,0.001051,0.000478,0.000203,0.000094,0.000908,0.000239
4,0063ab88.wav,0.000013,0.000033,0.000055,0.000007,0.000117,0.000507,0.001025,0.000040,0.991004,...,0.000075,0.000017,0.000111,0.000012,0.000005,0.000109,0.000440,0.000234,0.000135,0.000070
5,0075d39c.wav,0.000002,0.000062,0.000044,0.000015,0.000018,0.000007,0.000022,0.000035,0.000027,...,0.000014,0.000043,0.000024,0.000043,0.000018,0.000026,0.000001,0.000039,0.000071,0.000003
6,00780200.wav,0.002488,0.001849,0.000449,0.000022,0.000105,0.000521,0.688182,0.000189,0.000277,...,0.000869,0.000345,0.000292,0.000682,0.002621,0.003435,0.000171,0.005600,0.007439,0.000425
7,0079d310.wav,0.000269,0.004312,0.000250,0.000308,0.003426,0.000971,0.001505,0.011514,0.414048,...,0.000449,0.000248,0.002400,0.000592,0.000088,0.002007,0.000103,0.000809,0.003556,0.020507
8,00ad7068.wav,0.000519,0.000937,0.003695,0.000478,0.000580,0.001373,0.000320,0.005693,0.000440,...,0.000291,0.000509,0.000430,0.000191,0.000224,0.000128,0.000920,0.000565,0.001126,0.000572
9,00c5808a.wav,0.000508,0.008090,0.002649,0.000416,0.001449,0.003935,0.004418,0.001659,0.484709,...,0.011046,0.000421,0.000679,0.001631,0.000896,0.010953,0.000639,0.000465,0.004119,0.012590


### Step6
* 要給其他人co-train的

In [36]:
type_ = 'mfcc7' #要抽取哪一個種類的unverified trainX出來去re-train
un_or_test = 'test' # unverified or test
folder = 'feature/'+type_+'/cotrain/'+un_or_test+'/' #共同predict對unverified data的結果


acc_df = pd.read_csv('feature/'+type_+'/semi/valid_acc_'+type_+'.csv')
acc_df.columns = ['unverified','test','acc']
acc_df = acc_df.filter([un_or_test,'acc'])
files = os.listdir(folder)

ratio_all=0
for i,csv in enumerate(files):
    if csv.startswith('validation_ACC'):
        continue
    else:
        ratio = acc_df[acc_df[un_or_test] == csv]['acc'].values[0]
#         print(ratio)
        ratio_all += ratio
#     df = pd.read_csv(os.path.join(folder,csv),header=None) #ori method
    df = pd.read_csv(os.path.join(folder,csv),header=0,index_col=0)
    if df.iloc[0,0] == 'fname':
        df = df.drop(0,axis=0)
#     df = df.drop(0,axis=1) #ori method
    df = df.drop(['fname'],axis=1)

    if i==0:
        train_X = df.values*ratio
    else:
        train_X += df.values*ratio 
print(train_X.shape)
train_X = train_X / ratio_all
print(sum(train_X[0]))

(9400, 41)
0.9999999876874105


In [37]:
df_test = pd.read_csv(os.path.join(folder,csv),header=0,index_col=0)
df_test = df_test.filter(['fname'])
# fname = np.load('feature/mfcc6/fname_unverified.npy')
# df_test = pd.DataFrame(fname,columns=['fname'])
# df = df[df['manually_verified']==0]
# df.reset_index(drop=True,inplace=True)
df_test['label_verified'] = "none"
df_test['verified_confidence']=0.0
# df_ = pd.DataFrame(columns=['label_verified','verified_confidence'])
for i,r in df.iterrows():
    df_test.iloc[i,1] = np.argmax(train_X[i])
    df_test.iloc[i,2] = max(train_X[i])
#     print(i)
#     print(r['label'])
df_test

,fname,label_verified,verified_confidence
0,00063640.wav,30,0.524728
1,0013a1db.wav,16,0.889711
2,002bb878.wav,21,0.425855
3,002d392d.wav,21,0.874751
4,00326aa9.wav,15,0.962463
5,0038a046.wav,34,0.472094
6,003995fa.wav,23,0.387229
7,005ae625.wav,38,0.382951
8,007759c4.wav,38,0.250561
9,008afd93.wav,1,0.966930


In [38]:
type_ = 'mfcc7' #要抽取哪一個種類的unverified trainX出來去re-train
un_or_test = 'unverified' # unverified or test
folder = 'feature/'+type_+'/cotrain/'+un_or_test+'/' #共同predict對unverified data的結果


acc_df = pd.read_csv('feature/'+type_+'/semi/valid_acc_'+type_+'.csv')
acc_df.columns = ['unverified','test','acc']
acc_df = acc_df.filter([un_or_test,'acc'])
files = os.listdir(folder)

ratio_all=0
for i,csv in enumerate(files):
    if csv.startswith('validation_ACC'):
        continue
    else:
        ratio = acc_df[acc_df[un_or_test] == csv]['acc'].values[0]
#         print(ratio)
        ratio_all += ratio
#     df = pd.read_csv(os.path.join(folder,csv),header=None) #ori method
    df = pd.read_csv(os.path.join(folder,csv),header=0,index_col=0)
    if df.iloc[0,0] == 'fname':
        df = df.drop(0,axis=0)
#     df = df.drop(0,axis=1) #ori method
    df = df.drop(['fname'],axis=1)

    if i==0:
        train_X = df.values*ratio
    else:
        train_X += df.values*ratio 
print(train_X.shape)
train_X = train_X / ratio_all
print(sum(train_X[0]))

(5763, 41)
0.9999999938307781


In [39]:
fname = np.load('feature/mfcc6/fname_unverified.npy')
df_un = pd.DataFrame(fname,columns=['fname'])
# df = df[df['manually_verified']==0]
# df.reset_index(drop=True,inplace=True)
df_un['label_verified'] = "none"
df_un['verified_confidence']=0.0
# df_ = pd.DataFrame(columns=['label_verified','verified_confidence'])
for i,r in df_un.iterrows():
    df_un.iloc[i,1] = np.argmax(train_X[i])
    df_un.iloc[i,2] = max(train_X[i])
#     print(i)
#     print(r['label'])
df_un

,fname,label_verified,verified_confidence
0,00044347.wav,0,0.674100
1,002d256b.wav,5,0.174969
2,003b91e8.wav,4,0.310127
3,004ad66f.wav,7,0.811045
4,0063ab88.wav,8,0.573831
5,0075d39c.wav,9,0.721046
6,00780200.wav,6,0.491106
7,0079d310.wav,24,0.421566
8,00ad7068.wav,12,0.925342
9,00c5808a.wav,13,0.416984


In [40]:
df_semi = df_un.append(df_test)
# df_semi

df_semi = df_semi.reset_index(drop=True)
df_semi

,fname,label_verified,verified_confidence
0,00044347.wav,0,0.674100
1,002d256b.wav,5,0.174969
2,003b91e8.wav,4,0.310127
3,004ad66f.wav,7,0.811045
4,0063ab88.wav,8,0.573831
5,0075d39c.wav,9,0.721046
6,00780200.wav,6,0.491106
7,0079d310.wav,24,0.421566
8,00ad7068.wav,12,0.925342
9,00c5808a.wav,13,0.416984


In [41]:
threshold_upper = df_semi['verified_confidence'].mean() + df_semi['verified_confidence'].std()
df_semi['take']=0
for i,r in df_semi.iterrows():
    if r['verified_confidence']>threshold_upper:
        df_semi.iloc[i,3]=1
    else:
        df_semi.iloc[i,3]=0
df_semi

,fname,label_verified,verified_confidence,take
0,00044347.wav,0,0.674100,0
1,002d256b.wav,5,0.174969,0
2,003b91e8.wav,4,0.310127,0
3,004ad66f.wav,7,0.811045,0
4,0063ab88.wav,8,0.573831,0
5,0075d39c.wav,9,0.721046,0
6,00780200.wav,6,0.491106,0
7,0079d310.wav,24,0.421566,0
8,00ad7068.wav,12,0.925342,1
9,00c5808a.wav,13,0.416984,0


In [42]:
df_ = df_semi[df_semi['take']==1]
df_ = df_.filter(['fname','label_verified'])
df_.to_csv('feature/'+type_+'/cotrain/Y_leo_cotrain_verified.csv',index=False) # 儲存train_Y
df_

,fname,label_verified
8,00ad7068.wav,12
11,00c9e799.wav,15
19,0120d246.wav,22
46,02274ee8.wav,28
49,0236dccb.wav,33
54,0266774c.wav,33
60,02d7fba3.wav,29
72,03657d6d.wav,30
75,0395ba61.wav,15
81,03e88255.wav,34


### step7 
## produce semi data
* 給自己co-train的

In [43]:
feature_type = 'mfcc7'
who = 'Y_selftrain_ens_verified'
co_or_self = 'selftrain' #cotrain or selftrain
dicts = pickle.load(open('data/map.pkl','rb'))

In [44]:
X_un = np.load('feature/'+feature_type+'/X_unverified.npy')
Y_un = np.load('feature/'+feature_type+'/fname_unverified.npy')
Y_un = pd.DataFrame(Y_un,columns=['fname'])
X_test = np.load('feature/'+feature_type+'/X_test.npy')
Y_test = pd.read_csv('data/sample_submission.csv').filter(['fname'])
print(X_un.shape,X_test.shape)

(5763, 60, 259, 1) (9400, 60, 259, 1)


In [45]:
X_semi = np.concatenate((X_un,X_test))
print(X_semi.shape)
Y_semi = Y_un.append(Y_test)
len(Y_semi)

(15163, 60, 259, 1)


15163

In [46]:
Y_co = pd.read_csv(glob.glob('feature/'+feature_type+'/'+co_or_self+'/'+who+'.csv')[0]) #change如果有第二個的話要改
Y_semi = pd.merge(Y_semi,Y_co,how='left',on='fname')
Y_semi['label_verified'] = Y_semi['label_verified'].map(dicts)
Y_semi

,fname,label_verified
0,00044347.wav,NaN
1,002d256b.wav,NaN
2,003b91e8.wav,4.0
3,004ad66f.wav,NaN
4,0063ab88.wav,NaN
5,0075d39c.wav,9.0
6,00780200.wav,NaN
7,0079d310.wav,NaN
8,00ad7068.wav,12.0
9,00c5808a.wav,NaN


In [47]:
Y_semi = Y_semi.dropna()
Y_co_li = Y_semi.index.tolist()
X_co = X_semi[Y_co_li]
Y_co = np.array(Y_semi['label_verified']).astype('int')
len(Y_co_li)

4823

In [48]:
print(X_co.shape,Y_co.shape)
if not os.path.exists('feature/'+feature_type+'/semi/'+who+'/'):
    os.makedirs('feature/'+feature_type+'/semi/'+who+'/')
np.save('feature/'+feature_type+'/semi/'+who+'/X_co.npy',X_co)
np.save('feature/'+feature_type+'/semi/'+who+'/Y_co.npy',Y_co)


(4823, 60, 259, 1) (4823,)
